# 1. Hyper-parameters for experiment
- seed: controls synthetic data generation, and model reproduction
- datatype: input data type, options for synthetic data are 'XOR', 'nonlinear_additive', 'fusion_feature'
- mode: learning mode, options are 'active', 'passive'
- black_epochs: solve epoch of blackbox model
- solve_epochs: solve epoch of AID
- num_features: number of features you want to keep in interpreted model

In [1]:
### hyper-parameters for experiment
seed = 0
datatype = 'fusion_feature'
mode = 'passive'
black_epochs = 10
solve_epochs = 10
num_features = 4

# 2. Read data and generate prediction
you can replace follows to your own data and prediction

In [2]:
from util import *
from BlackBox import *
### load data
x_train, x_val, y_train, y_val = load_data(datatype=datatype, seed=seed)
### generate pred, you can replace pred_train, pred_val from you own model
blackbox = BlackBox(datatype=datatype, epochs=black_epochs)
blackbox.load(x_train, x_val, y_train, y_val)
blackbox.train()
m_dim = x_val.shape[1]
n_targets = y_val.shape[1]
pred_train, pred_val = blackbox.gen_preds_new(x_train, x_val)

/home/heartvoice/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/heartvoice/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/heartvoice/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/heartvoice/anaconda3/lib/pyth

Loading Data for Black-Box Model ...
x_train shape: (9000, 10)
y_train shape: (9000, 3)
x_val shape: (1000, 10)
y_val shape: (1000, 3)
Build Black-Box Model...
Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 2s - loss: 1.5667 - acc: 0.6548 - val_loss: 1.1757 - val_acc: 0.7450
Epoch 2/10
9000/9000 [==============================] - 1s - loss: 1.0597 - acc: 0.8176 - val_loss: 0.8201 - val_acc: 0.9600
Epoch 3/10
9000/9000 [==============================] - 1s - loss: 0.8176 - acc: 0.8808 - val_loss: 0.6231 - val_acc: 0.9510
Epoch 4/10
9000/9000 [==============================] - 1s - loss: 0.6975 - acc: 0.9022 - val_loss: 0.5300 - val_acc: 0.9620
Epoch 5/10
9000/9000 [==============================] - 1s - loss: 0.6330 - acc: 0.9146 - val_loss: 0.4771 - val_acc: 0.9800
Epoch 6/10
9000/9000 [==============================] - 1s - loss: 0.6029 - acc: 0.9190 - val_loss: 0.4454 - val_acc: 0.9880
Epoch 7/10
9000/9000 [====================

In [3]:
import dill
### all the input data
input_data = {}
input_data['x_train'] = x_train
input_data['y_train'] = y_train
input_data['pred_train'] = pred_train
input_data['x_val'] = x_val
input_data['y_val'] = y_val
input_data['pred_val'] = pred_val
with open('res/blackbox_pred.pkl', 'wb') as fout:
    dill.dump(input_data, fout)

# 3. Run for AID

In [4]:
from AID import SolveNN
opt = SolveNN(l=num_features, m_dim=m_dim, n_targets=n_targets, blackbox_pred_path='res/blackbox_pred.pkl', epochs=solve_epochs, seed=seed, mode=mode, datatype=datatype, eval_more=False)
opt.load(None, None, None, None)
opt.train()
res = opt.get_res()

Initializing Explainer ...
Loading Data ...
x_train shape: (9000, 10)
y_train shape: (9000, 3)
x_val shape: (1000, 10)
y_val shape: (1000, 3)
epoch 1
mean ROC-AUC: 0.6940, mean PR-AUC: 0.5307, mean fidelity: 0.7103, mean accuracy: 0.7047
mean ROC-AUC: 0.9981, mean PR-AUC: 0.9970, mean fidelity: 1.0000, mean accuracy: 0.9837
epoch 2
mean ROC-AUC: 0.7294, mean PR-AUC: 0.5769, mean fidelity: 0.7520, mean accuracy: 0.7457
mean ROC-AUC: 0.9981, mean PR-AUC: 0.9970, mean fidelity: 1.0000, mean accuracy: 0.9837
epoch 3
mean ROC-AUC: 0.7498, mean PR-AUC: 0.6020, mean fidelity: 0.7603, mean accuracy: 0.7567
mean ROC-AUC: 0.9981, mean PR-AUC: 0.9970, mean fidelity: 1.0000, mean accuracy: 0.9837
epoch 4
mean ROC-AUC: 0.7619, mean PR-AUC: 0.6156, mean fidelity: 0.7663, mean accuracy: 0.7633
mean ROC-AUC: 0.9981, mean PR-AUC: 0.9970, mean fidelity: 1.0000, mean accuracy: 0.9837
epoch 5
mean ROC-AUC: 0.7737, mean PR-AUC: 0.6246, mean fidelity: 0.7663, mean accuracy: 0.7640
mean ROC-AUC: 0.9981, mean

# 4. Get Interpretation Results

In [5]:
scores = opt.scores
print('Interpreted scores has shape {0}, which is {1} samples, {2} features and {3} targets.'.format(scores.shape, scores.shape[0], scores.shape[1], scores.shape[2]))

Interpreted scores has shape (1000, 10, 3), which is 1000 samples, 10 features and 3 targets.
